# Resume Parser Based Image File

In [1]:
import cv2
import numpy as np
import imutils
import pytesseract
import pdf_to_image
import os
import time
import shutil

In [4]:
def isloate_text(filename):
    os.mkdir('pages')
    path = 'data/' + filename
    if filename[-3:] == 'pdf' or filename[-3:] == 'PDF':
        pdf_to_image.pdfToImages(path)
    else:
        img = cv2.imread(path)
        cv2.imwrite('pages/' + filename, img)
    i = 1
    for page in os.listdir('pages/'):
        path = 'pages/' + page
        rgb = cv2.imread(path)
        rgb = imutils.resize(rgb, width = int(rgb.shape[1]/4))
        small = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)

        _, bw = cv2.threshold(small, 0.0, 255.0, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))
        connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)

        contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        with open('headers.txt') as f:
            lines = f.read().upper().splitlines()

        header_cords = []

        for idx in range(len(contours)):
            x, y, w, h = cv2.boundingRect(contours[idx])
            cv2.rectangle(rgb, (x, y, x+w, y+h), (0, 0, 255), 2)
            try:
                cropped = rgb[y-5:y+h+5, x-5:x+w+5]
                text = pytesseract.image_to_string(cropped)
            except:
                continue
            text = text.lower()
            print(text)
            if text in lines or text[:-1] in lines:
                if text[len(text) - 1] != '.':
                    header_cords.append([x, y, x+w, y+h])
        cv2.imwrite('result/' + filename[:-4] + '-' + str(i) + '.jpg', rgb)
        i = i + 1
    shutil.rmtree('./pages/')
    return header_cords

In [5]:
total = 0
count = 0
for each in os.listdir('data/'):
    start = time.time()
    print('Working on: ' + each)
    isloate_text(each)
    total_time = time.time() - start
    print('Time taken: ' + str(total_time) + ' seconds')
    total = total + total_time
    count = count + 1
print('Total time taken: ' + str(total) + ' seconds')
print('Average time taken: ' + str(total/count) + ' seconds')

Working on: Programmer-Resume-Sample.jpg

 








 

























Time taken: 6.89665412902832 seconds
Working on: Net-Developer-Resume-Sample.jpg



















 








 

Time taken: 4.849189519882202 seconds
Working on: IT-Project-Manager-Resume-Template.jpg

 




































 








Time taken: 8.160841464996338 seconds
Working on: Web-Designer-Resume-Sample.jpg








 










 




Time taken: 3.698066473007202 seconds
Working on: Entry-Level-Software-Engineer-Resume-Sample-1.jpg



 







 


 





 




 


Time taken: 3.768622636795044 seconds
Working on: Network-Administrator-Resume-Sample-Template.jpg




 

































Time taken: 6.368748426437378 seconds
Working on: Information-Technology-Resume-Example-Template.jpg
 


# Resume Parser Based on PDF or Docx File with GUI

In [6]:
import os
import flask
import pickle
import sys, fitz
from flask import Flask, request, url_for, render_template, send_from_directory
from itertools import *
import time
import random

import spacy
import sqlite3

In [7]:
UPLOAD_FOLDER = os.path.dirname(os.path.abspath("__file__")) + '/uploads/'

In [8]:
model_path = 'model/model_2'
model = spacy.load(model_path)

app = Flask(__name__, static_url_path="/static")
DIR_PATH = os.path.dirname(os.path.abspath("__file__"))
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [9]:
@app.route('/uploads/<filename>')
def uploaded_file(filename):
   return send_from_directory(app.config['DOWNLOAD_FOLDER'], filename, as_attachment=True)

@app.route('/', methods=['GET', 'POST'])
def main():
    if request.method == 'POST':
        f = request.files.getlist('files')
        result = []

        for file in f:
            if file:
                filename = file.filename
                file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

                upload_dir = os.path.join(app.config['UPLOAD_FOLDER'], filename)
                
                doc = fitz.open(upload_dir)
                text = ""
                for page in doc:
                    text = text + str(page.getText())
                
                tx = "".join(text.split('\n'))
                doc = model(tx)
                # entities = {key: list(g) for key, g in groupby(sorted(doc.ents, key=lambda x: x.label_), lambda x: x.label_)}
                entities = {key: list(set(map(lambda x: str(x), g))) for key, g in groupby(sorted(doc.ents, key=lambda x: x.label_), lambda x: x.label_)}

                result.append(entities)

        return render_template('result.html', result=result)

    return render_template('index.html')

In [10]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/May/2021 10:32:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 10:32:15] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 10:32:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 10:32:44] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 10:32:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 10:32:56] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 10:33:02] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 10:33:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 10:33:05] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 10:33:11] "POST / HTTP/1.1" 200 -
